# Read Data

转换成字符层级的 带有空格

In [1]:
from collections import defaultdict
len_counter = defaultdict(int)
language = 'chinese'
#language = 'english'
print(language)
if language == 'chinese':
    workpath = '/ldev/tensorflow/nmt2/nmt/data/wordlevel/'
    out_path = '/ldev/tensorflow/nmt2/nmt/data/charspace/'
    min_fre = 100
elif language == 'english':
    workpath = '/ldev/tensorflow/nmt2/nmt/data/enwordlevel/'
    out_path = '/ldev/tensorflow/nmt2/nmt/data/encharspace/'
    min_fre = 80

chinese


In [2]:
def sentece2char_withspace(sentence):
    words = sentence.split()
    chars = []
    for i,word in enumerate(words):
        for char in word:
            chars.append(char)
        if i < len(words) - 1:
            chars.append('<SPACE>')
    return chars
print(sentece2char_withspace('嗷嗷 大 湿的 左手 在 干嘛 ， 看 着 小 纯洁 撸么 。'))


['嗷', '嗷', '<SPACE>', '大', '<SPACE>', '湿', '的', '<SPACE>', '左', '手', '<SPACE>', '在', '<SPACE>', '干', '嘛', '<SPACE>', '，', '<SPACE>', '看', '<SPACE>', '着', '<SPACE>', '小', '<SPACE>', '纯', '洁', '<SPACE>', '撸', '么', '<SPACE>', '。']


# 统计分布

In [3]:
#要保证顺序对，这样可以不用再次读入
set_types = ['dev','test','train']
pair_types = ['response','message']

messages = []
responses = []

for st in set_types:
    for pt in pair_types:
        if pt == 'response':
            container = responses
        else:
            container = messages
        with open("%s%s.%s" % (workpath,st,pt),'r',encoding='utf-8') as fin:
            lines = fin.readlines()
            for line in lines:
                container.append(sentece2char_withspace(line.strip('\n')))
print(len(messages))

3788600


In [4]:
# Statistics
from collections import defaultdict
len_counter = defaultdict(int)
word_len_counter = defaultdict(int)
num_of_words = 0
tokenized_messages = messages
tokenized_responses = responses
for line in tokenized_messages + tokenized_responses:
    len_counter[len(line)] += 1
    num_of_words += len(line)
    for item in line:
        word_len_counter[len(item)] += 1

sorted_length = sorted(len_counter.items(), key = lambda x:x[0])
total_counter = 0
print('tokens per sentence:')
for lens,counter in sorted_length:
    total_counter += counter
    print('%d %.2f' % (lens, total_counter/len(tokenized_messages)/2))

print('char_nums_per_word:')
sorted_length = sorted(word_len_counter.items(), key = lambda x:x[0])
total_counter = 0
for lens,counter in sorted_length:
    total_counter += counter
    print('%d %.2f' % (lens, total_counter/num_of_words))

tokens per sentence:
3 0.00
4 0.00
5 0.00
6 0.00
7 0.00
8 0.00
9 0.00
10 0.00
11 0.00
12 0.01
13 0.02
14 0.05
15 0.08
16 0.12
17 0.16
18 0.20
19 0.24
20 0.28
21 0.32
22 0.36
23 0.40
24 0.43
25 0.47
26 0.50
27 0.53
28 0.57
29 0.60
30 0.63
31 0.66
32 0.68
33 0.71
34 0.74
35 0.76
36 0.79
37 0.81
38 0.83
39 0.85
40 0.87
41 0.89
42 0.91
43 0.93
44 0.95
45 0.96
46 0.97
47 0.98
48 0.99
49 0.99
50 0.99
51 0.99
52 1.00
53 1.00
54 1.00
55 1.00
56 1.00
57 1.00
58 1.00
59 1.00
60 1.00
61 1.00
62 1.00
63 1.00
64 1.00
65 1.00
66 1.00
67 1.00
68 1.00
69 1.00
70 1.00
71 1.00
72 1.00
73 1.00
74 1.00
75 1.00
76 1.00
77 1.00
78 1.00
79 1.00
80 1.00
81 1.00
82 1.00
83 1.00
84 1.00
85 1.00
86 1.00
87 1.00
88 1.00
89 1.00
90 1.00
91 1.00
92 1.00
93 1.00
char_nums_per_word:
1 0.62
7 1.00


In [5]:
file_map = {'message':messages, 'response':responses}
dev_num = 20000
test_num = 20000
total_num = len(messages)
for file_type in file_map.keys():
    container = file_map[file_type]
    with open(out_path + 'dev.'+file_type,'w+',encoding='utf-8') as fout:
        for i in range(0,dev_num):
            fout.write('%s\n' % ' '.join(container[i]))
    with open(out_path + 'test.'+file_type,'w+',encoding='utf-8') as fout:
        for i in range(dev_num,test_num + test_num):
            fout.write('%s\n' % ' '.join(container[i]))
    with open(out_path + 'train.'+file_type,'w+',encoding='utf-8') as fout:
        for i in range(test_num + test_num,total_num):
            fout.write('%s\n' % ' '.join(container[i]))

# Create Vocab

In [6]:
in_path = out_path

from collections import defaultdict
counter = defaultdict(int)
message_counter = defaultdict(int)
response_counter = defaultdict(int)


for file in ['message','response']:
    with open(in_path + 'train.'+file,'r+',encoding='utf-8') as fin:
        lines = fin.readlines()
        container = message_counter
        if file == 'response':
            container = response_counter
        for line in lines:
            chars = line.strip('\n').split(' ')
            for char in chars:
                if len(char) > 0:
                    counter[char] += 1
                    container[char] += 1

sorted_counter = sorted(counter.items(), key = lambda x:x[1], reverse = True)
sorted_message_counter = sorted(message_counter.items(), key = lambda x:x[1], reverse = True)
sorted_response_counter = sorted(response_counter.items(), key = lambda x:x[1], reverse = True)

In [7]:
print(len(sorted_message_counter))
filtered_sorted_message_counter = list()
for item in sorted_message_counter:
    if (item[1] >= min_fre):
        filtered_sorted_message_counter.append(item)
print(len(filtered_sorted_message_counter))

print(len(sorted_response_counter))
filtered_sorted_response_counter = list()
for item in sorted_response_counter:
    if (item[1] >= min_fre):
        filtered_sorted_response_counter.append(item)
print(len(filtered_sorted_response_counter))

6689
4328
10550
4082


In [8]:

vocab_nums = [40000]
for vocab_num in vocab_nums:
    for file in ['message','response']:
        with open(out_path + 'vocab.'+str(vocab_num)+'.'+file,'w+',encoding='utf-8') as fout:
            #fout.write('<UNK>\n<S>\n</S>\n');
            for item in sorted_counter[0:vocab_num]:
                fout.write(item[0]+'\n')
    with open(out_path + 'vocab.'+str(vocab_num)+'.separate.response','w+',encoding='utf-8') as fout:
            #fout.write('<UNK>\n<S>\n</S>\n');
            for item in filtered_sorted_response_counter[0:vocab_num]:
                fout.write(item[0]+'\n')
    with open(out_path + 'vocab.'+str(vocab_num)+'.separate.message','w+',encoding='utf-8') as fout:
            #fout.write('<unk>\n<s>\n</s>\n');
            for i,item in enumerate(filtered_sorted_message_counter[0:vocab_num]):
                if len(item[0]) == 0:
                    print('flag '+str(i))
                fout.write(item[0]+'\n')

    with open(out_path + 'vocab.'+str(vocab_num)+'.counter','w+',encoding='utf-8') as fout:
            fout.write('<unk>\n\t-1\n<s>\t-1\n</s>\t-1\n');
            for item in sorted_counter[0:vocab_num]:
                fout.write('%s\t%d\n'  % (item[0],item[1]))

In [9]:
sums = 0
for i,(lens,counter) in enumerate(sorted_message_counter):
    sums += counter
print(len(sorted_message_counter))
total_counter = 0
for i,(lens,counter) in enumerate(sorted_message_counter):
    total_counter += counter
    if i%10000 == 0:
        print('%d %.4f' % (i, total_counter/sums))

6689
0 0.3768


##### Segment

In [18]:
# Configs including a end
seg_len = 5
seg_end = '<#>'
seg_pad = '<_>'
seg_separator = '\t'
seg_inter_separator = ' '
vocab_nums = [20000,40000]
from collections import defaultdict
# read the vocabs and add all characters in the vocab
def add_chars_to_vocab(file_path,vocab_path, word_num = -1):
    with open(file_path,'r+',encoding='utf-8') as fin:
        with open(vocab_path+'_tmp','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = defaultdict(int)
            vocab[seg_end] = 999
            vocab[seg_pad] = 999
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                line=line.replace(' ','')
                line=line.replace('\t','')
                #vocab.add(word)
                if len(line) > 1:
                    for char in line:
                        vocab[char] += 1
                
            for token in vocab.keys():
                if vocab[token] > 20:
                    fout.write(token+'\n')
                #print(token)
            print('new num: %d' % len(vocab))
    with open(vocab_path+'_tmp','r+',encoding='utf-8') as fin:
        with open(vocab_path+'_seg','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = set()
            vocab.add(seg_end)
            vocab.add(seg_pad)
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                #vocab.add(word)
                if len(line) > 0:
                    for char in line:
                        vocab.add(str(char))
            count = 0
            for token in vocab:
                if len(token) > 0:
                    fout.write(token+'\n')
                    count += 1
                #print(token)
            print('new num: %d' % count)



def convert_to_seg_file(file_path, seg_len,vocab):
    with open(file_path, 'r+', encoding='utf-8') as fin:
        if file_path[-len('message'):] == 'message':
            nfile_path = file_path[0:-len('message')]+''+str(vocab)+'.message'
        else:
            nfile_path = file_path[0:-len('response')]+''+str(vocab)+'.response'
        with open(nfile_path , 'w+', encoding='utf-8') as foout,open(nfile_path + '_seg', 'w+', encoding='utf-8') as fout,open(nfile_path + '_seg_len', 'w+', encoding='utf-8') as flout:
            lines = fin.readlines()
            for line in lines:
                foout.write(line)
                items = line.strip('\n').split(' ')
                seg_items = []
                for item in items:
                    item = list(item)
                    item = item[0:seg_len - 1]
                    item.append(seg_end)
                    while len(item) != seg_len:
                        item.append(seg_pad)
                    seg_items.append(seg_inter_separator.join(item))
                flout.write(' '.join([str(min(seg_len,len(x)+1)) for x in items]) + '\n')
                fout.write(seg_separator.join(seg_items) + '\n')

for vocab in vocab_nums:
    # TODO 所有
    add_chars_to_vocab(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/train.message', r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/vocab.%d.separate.message' % vocab)
    add_chars_to_vocab(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/train.response', r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/vocab.%d.separate.response'% vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/train.message', seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/train.response', seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/test.message', seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/test.response', seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/dev.message', seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/wordlevel/dev.response', seg_len,vocab)



original num: 3748600
new num: 6690
original num: 5593
new num: 5593
original num: 3748600
new num: 10551
original num: 5572
new num: 5572
original num: 3748600
new num: 6690
original num: 5593
new num: 5593
original num: 3748600
new num: 10551
original num: 5572
new num: 5572
